In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import re
import pandas as pd
import joblib
import numpy as np
import xgboost as xgb

xgb.set_config(verbosity=2, use_rmm=True)


In [3]:

# dataset_dir = 'sentiment140'
dataset_dir = 'imdb'
# dataset_dir = 'coronaNLP'

# n_gram = (1, 1)
# n_gram = (1, 2)
n_gram = (2, 2)

# importing the processed dataframe
df = joblib.load(f'./dataframes/df_{dataset_dir}.pkl')

df.head()


,review,sentiment
0,one review mention watch 1 oz episod youll hoo...,1
1,wonder littl product br br film techniqu veri ...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic famili littl boy jake think zombi closet...,0
4,petter mattei love time money visual stun film...,1


In [4]:


X = df.iloc[:, 0]


y = df.iloc[:, 1]

X, y


(0        one review mention watch 1 oz episod youll hoo...
 1        wonder littl product br br film techniqu veri ...
 2        thought wonder way spend time hot summer weeke...
 3        basic famili littl boy jake think zombi closet...
 4        petter mattei love time money visual stun film...
                                ...                        
 49995    thought movi right good job wasnt creativ orig...
 49996    bad plot bad dialogu bad act idiot direct anno...
 49997    cathol taught parochi elementari school nun ta...
 49998    im go disagre previous comment side maltin one...
 49999    one expect star trek movi high art fan expect ...
 Name: review, Length: 49582, dtype: object,
 0        1
 1        1
 2        1
 3        0
 4        1
         ..
 49995    1
 49996    0
 49997    0
 49998    0
 49999    0
 Name: sentiment, Length: 49582, dtype: int32)

In [5]:

tfidf = joblib.load(
    f"./vectors/vectorizer_{dataset_dir}_{n_gram}.pkl")
tfidf


TfidfVectorizer(max_features=143417, ngram_range=(2, 2),
                tokenizer=<bound method RegexpTokenizer.tokenize of RegexpTokenizer(pattern='[a-zA-Z0-9]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>)

In [6]:
tfidf.vocabulary_, tfidf.idf_


({'one review': 90573,
  'review mention': 104914,
  'mention watch': 80225,
  'watch 1': 135625,
  'episod youll': 39203,
  'youll hook': 143020,
  'exact happen': 41357,
  'mebr br': 79718,
  'br first': 17046,
  'first thing': 48655,
  'thing struck': 125378,
  'scene violenc': 108451,
  'set right': 112550,
  'right word': 105426,
  'word go': 140069,
  'faint heart': 43229,
  'pull punch': 100237,
  'drug sex': 35210,
  'sex violenc': 112852,
  'wordbr br': 140147,
  'br call': 16634,
  'maximum secur': 79137,
  'focus main': 49274,
  'prison cell': 99109,
  'front face': 50664,
  'never far': 86783,
  'awaybr br': 10013,
  'br would': 18338,
  'would say': 141553,
  'say main': 107409,
  'main appeal': 76661,
  'show due': 113897,
  'due fact': 35278,
  'goe show': 54641,
  'show wouldnt': 114359,
  'wouldnt dare': 141708,
  'pretti pictur': 98940,
  'pictur paint': 95532,
  'mainstream audienc': 76746,
  'mess around': 80282,
  'around first': 8351,
  'first episod': 48406,
  'e

In [7]:
X = tfidf.transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape


((39665, 143417), (39665,))

In [9]:
# creating our pipeline that will return an estimator
pipeline = Pipeline([('clf', xgb.XGBClassifier(use_label_encoder=False))])

parameters = {
    'clf__booster': ('gbtree', 'gblinear', 'dart'),
    'clf__eta': (0, 0.5, 1),
    }

clf = GridSearchCV(pipeline, param_grid=parameters, scoring='accuracy', cv=5, verbose=True)
clf


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('clf',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      enable_categorical=False,
                                                      gamma=None, gpu_id=None,
                                                      importance_type=None,
                                                      interaction_constraints=None,
                                                      learning_rate=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                         

In [10]:

clf = clf.fit(X_train, y_train)


Fitting 5 folds for each of 9 candidates, totalling 45 fits


C:\Users\Enes\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:34:08] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=6
[18:34:08] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:34:09] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=6
[18:34:09] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=6
[18:34:09] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=6
[18:34:09] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 pruned nodes, max

C:\Users\Enes\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:34:29] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[18:34:29] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:34:30] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[18:34:30] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[18:34:30] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[18:34:30] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max

...

[18:50:31] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[18:50:31] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\gbm\gbtree.cc:904: drop 0 trees, weight = 1
[18:50:31] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:50:31] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=6
[18:50:31] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\gbm\gbtree.cc:904: drop 0 trees, weight = 1
[18:50:32] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=6
[18:50:32] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\gbm\gbtree.cc:904: drop 0

C:\Users\Enes\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:51:10] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[18:51:10] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\gbm\gbtree.cc:904: drop 0 trees, weight = 1
[18:51:10] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:51:11] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=6
[18:51:11] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\gbm\gbtree.cc:904: drop 0 trees, weight = 1
[18:51:11] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\tree\updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=6
[18:51:11] INFO: D:\Build\xgboost\xgboost-1.5.1.git\src\gbm\gbtree.cc:904: drop 0

C:\Users\Enes\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:51:49] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [11]:


y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.83      0.83      0.83      4939
           1       0.83      0.83      0.83      4978

    accuracy                           0.83      9917
   macro avg       0.83      0.83      0.83      9917
weighted avg       0.83      0.83      0.83      9917



In [12]:


print("Best: %f using %s" % (clf.best_score_, 
    clf.best_params_))
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.836254 using {'clf__booster': 'gblinear', 'clf__eta': 0.5}
0.498147 (0.000050) with: {'clf__booster': 'gbtree', 'clf__eta': 0}
0.786336 (0.002965) with: {'clf__booster': 'gbtree', 'clf__eta': 0.5}
0.786083 (0.005382) with: {'clf__booster': 'gbtree', 'clf__eta': 1}
0.498147 (0.000050) with: {'clf__booster': 'gblinear', 'clf__eta': 0}
0.836254 (0.003216) with: {'clf__booster': 'gblinear', 'clf__eta': 0.5}
0.775369 (0.010325) with: {'clf__booster': 'gblinear', 'clf__eta': 1}
0.498147 (0.000050) with: {'clf__booster': 'dart', 'clf__eta': 0}
0.786336 (0.002965) with: {'clf__booster': 'dart', 'clf__eta': 0.5}
0.786083 (0.005382) with: {'clf__booster': 'dart', 'clf__eta': 1}


In [13]:




from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))

acc = int(accuracy_score(y_test, y_pred)*100)



[[4101  838]
 [ 831 4147]]


In [14]:


test_tweet = "groceri store"
vector = tfidf.transform([test_tweet])

print(clf.predict(vector))



[0]


In [16]:



# exporting the pipeline
joblib.dump(clf.best_estimator_, f'./models/xgb_{dataset_dir}_{acc}_{n_gram}.pkl')




['./models/xgb_imdb_83_(2, 2).pkl']